In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='astute-baton-424012-q7')

In [3]:
# Load the satellite image
image = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')

# Select the RGB bands
rgb = image.select(['R', 'G', 'B'])

# Visualization parameters
rgbVis = {
  'min': 11.0,
  'max': 190.0,
}

# Set map center and zoom level
center = [50.825, 3.264]

# Create a map
Map = geemap.Map(center=center, zoom=19)

# Add satellite basemap
Map.add_basemap('Satellite')

# Add the image to the map
Map.addLayer(rgb, rgbVis, 'RGB Image')

# Display the map
Map


Map(center=[50.825, 3.264], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [4]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Add Earth Engine dataset
dem = ee.Image("USGS/SRTMGL1_003")
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select("landcover")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

# Add Earth Engine layers to Map
Map.addLayer(dem, vis_params, "SRTM DEM", True, 0.5)
Map.addLayer(landcover, {}, "Land cover")
Map.addLayer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 1.5},
    "Landsat 7",
)
Map.addLayer(states, {}, "US States")

In [6]:
download_dir= "./exported_images/"

In [7]:
import os
png_file = os.path.join(download_dir, "my_map.png")

In [8]:
Map.to_image(filename=png_file, monitor=1)

In [9]:
jpg_file = os.path.join(download_dir, "my_map.jpg")

In [10]:
Map.to_image(filename=jpg_file, monitor=1)

In [13]:
# Create a map
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)

# Load Earth Engine datasets
dem = ee.Image("USGS/SRTMGL1_003")
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select("landcover")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"]
}

# Add Earth Engine layers to the map
Map.addLayer(dem, vis_params, "SRTM DEM", True, 0.5)
Map.addLayer(landcover, {}, "Land cover")
Map.addLayer(landsat7, {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 1.5}, "Landsat 7")
Map.addLayer(states, {}, "US States")

# Define the directory and filename to save the image
download_dir = "./exported_images"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
jpg_file = os.path.join(download_dir, "my_map.jpg")

# Save the map as an image
Map.to_image(filename=jpg_file, monitor=1)

In [15]:
import requests
import ee

img = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')
region = ee.Geometry.BBox(-122.0859, 37.0436, -122.0626, 37.0586)

# Multi-band GeoTIFF file.
url = img.getDownloadUrl({
    'bands': ['B3', 'B8', 'B11'],
    'region': region,
    'scale': 20,
    'format': 'GEO_TIFF'
})
response = requests.get(url)
with open('multi_band.tif', 'wb') as fd:
  fd.write(response.content)

In [64]:
import ee
import geemap
import os
from PIL import Image
import time


# Load the satellite image collection and filter by date
image_collection = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')

# Check if the image collection contains any images
count = image_collection.size().getInfo()
print(f'Number of images in collection: {count}')

if count == 0:
    raise ValueError("The image collection is empty. Adjust the date range or dataset.")

# Print available bands in the first image
first_image = image_collection.first()
bands = first_image.bandNames().getInfo()
print(f'Available bands: {bands}')

# Select the RGB bands if available
if 'R' in bands and 'G' in bands and 'B' in bands:
    rgb = image_collection.select(['R', 'G', 'B']).mosaic()
else:
    raise ValueError("The expected RGB bands are not available in the image collection.")

# Visualization parameters
rgbVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
    'gamma': 1.4
}

# Set map center and zoom level
center = [50.825, 3.264]

# Define the area of interest (AOI)
aoi = ee.Geometry.Point(center).buffer(1000)  # Buffer of 1000 meters around the center point

# Clip the image to the AOI
clipped_image = rgb.clip(aoi)

# Apply visualization parameters to the image
visualized_image = clipped_image.visualize(**rgbVis)

# Export the visualized image to a local directory with a larger scale to reduce the file size
local_folder = 'output_images'
os.makedirs(local_folder, exist_ok=True)
tif_file_path = os.path.join(local_folder, 'RGB_Image.tif')

# Use geemap to export the image
geemap.ee_export_image(
    visualized_image,  # Pass the visualized image as the first argument
    filename=tif_file_path,
    scale=1,  # Set a smaller scale to capture details
    region=aoi.bounds().getInfo()['coordinates'],
    crs='EPSG:4326'
)

# Wait for the file to be written
time.sleep(10)

# Verify the file was created
if os.path.exists(tif_file_path):
    print('Image exported as TIF.')

    # Convert the TIF image to JPG
    jpg_file_path = os.path.join(local_folder, 'RGB_Image.jpg')

    # Open the TIF image and convert it to JPG
    with Image.open(tif_file_path) as img:
        img.convert('RGB').save(jpg_file_path, 'JPEG')

    print(f'Image converted and saved as: {jpg_file_path}')
else:
    print(f'Failed to export the image. TIF file not found at {tif_file_path}.')


Number of images in collection: 1094
Available bands: ['R', 'G', 'B']
Generating URL ...
Please wait ...
Data downloaded to /Users/liendejong/code/Industry_project/Industry-Project-Claerhout/api/output_images/RGB_Image.tif
Image exported as TIF.
Image converted and saved as: output_images/RGB_Image.jpg


In [31]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import BytesIO

# Initialize the Earth Engine module.
ee.Initialize()

# Define the center coordinate and radius (in kilometers)
center_lat, center_lon = 50.825, 3.264
radius_km = 2  # Example radius in kilometers

# Convert radius to degrees (approximately)
radius_deg = radius_km / 111.32  # 1 degree latitude ~ 111.32 km

# Define the bounding box
min_lon = center_lon - radius_deg
min_lat = center_lat - radius_deg
max_lon = center_lon + radius_deg
max_lat = center_lat + radius_deg

# Define grid size in degrees (approx. half a kilometer)
grid_size_deg = 0.5 / 111.32

# Function to generate a range of floats
def frange(start, stop, step):
    while start < stop:
        yield start
        start += step

# Create a grid of boxes
lons = list(frange(min_lon, max_lon, grid_size_deg))
lats = list(frange(min_lat, max_lat, grid_size_deg))

# Display a few images to verify
display_count = 0
max_display = 3

for lon in lons:
    for lat in lats:
        if display_count >= max_display:
            break
        
        geom = ee.Geometry.Rectangle([lon, lat, lon + grid_size_deg, lat + grid_size_deg])
        
        # Use the Landsat 8 collection
        image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(geom) \
            .filterDate('2023-01-01', '2023-12-31') \
            .sort('CLOUDY_PIXEL_PERCENTAGE', True)
        
        # Get the first image from the collection
        image = image_collection.first()

        # If no image is found, skip this cell
        if image.getInfo() is None:
            print(f"No image available for grid cell at {lon}, {lat}")
            continue

        # Clip the image to the geometry
        clipped_image = image.clip(geom)
        
        # Select the RGB bands (Landsat 8 bands are B4, B3, B2 for RGB)
        rgb = clipped_image.select(['B4', 'B3', 'B2'])
        rgbVis = {'min': 0, 'max': 3000}

        # Get the URL for the image download
        url = rgb.getThumbURL({'region': geom, 'dimensions': '500x500', 'format': 'png'})
        
        # Display the image
        response = requests.get(url)
        img = plt.imread(BytesIO(response.content), format='png')
        
        plt.figure()
        plt.imshow(img)
        plt.title(f'Grid Cell: {lon}, {lat}')
        plt.axis('off')
        
        display_count += 1

plt.show()


No image available for grid cell at 3.2460337765001794, 50.807033776500184
No image available for grid cell at 3.2460337765001794, 50.81152533237514
No image available for grid cell at 3.2460337765001794, 50.8160168882501
No image available for grid cell at 3.2460337765001794, 50.82050844412505
No image available for grid cell at 3.2460337765001794, 50.82500000000001
No image available for grid cell at 3.2460337765001794, 50.82949155587497
No image available for grid cell at 3.2460337765001794, 50.83398311174992
No image available for grid cell at 3.2460337765001794, 50.83847466762488
No image available for grid cell at 3.2505253323751346, 50.807033776500184
No image available for grid cell at 3.2505253323751346, 50.81152533237514
No image available for grid cell at 3.2505253323751346, 50.8160168882501
No image available for grid cell at 3.2505253323751346, 50.82050844412505
No image available for grid cell at 3.2505253323751346, 50.82500000000001
No image available for grid cell at 3.

KeyboardInterrupt: 